In [5]:
import pandas as pd
import catboost
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor


In [6]:
data = pd.read_csv('yandex_data.tsv', sep='\t')
num_features = ['agent_fee', 'floor', 'floors_total', 'kitchen_area', 'living_area', 'price', 'rooms_offered', 'total_area', 'total_images']
cat_features = ['balcony', 'building_type', 'month', 'renovation', 'studio']

X_train, X_test, y_train, y_test = train_test_split(data[num_features], data['exposition_time'], test_size=0.3, shuffle=False) #+cat_features

1

In [ ]:
cclf = CatBoostRegressor()
cclf.fit(X_train, y_train)

In [ ]:
lclf = LGBMRegressor()
lclf.fit(X_train, y_train)

2

In [ ]:
gs = GridSearchCV(
    CatBoostRegressor(loss_function='MAE',  random_seed=0),
    param_grid={
        'max_depth': [5, 7, 9],
        'learning_rate': [0.05, 0.1, 0.5] ,
        
    }
)
gs.fit(X_train[num_features], y_train)

{'learning_rate': 0.1, 'max_depth': 9}

In [ ]:
mean_absolute_error(y_test, cclf.predict(X_test))

198390      7
198391     58
198392      1
198393     55
198394     83
         ... 
228385    830
228386    830
228387     91
228388      4
228389      4
Name: exposition_time, Length: 30000, dtype: int64

In [ ]:
from sklearn.model_selection import GridSearchCV
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

3

In [ ]:
gs = GridSearchCV(
    CatBoostRegressor(loss_function='MAE',  random_seed=0),
    param_grid={
        'max_depth': [5, 7, 9],
        'learning_rate': [0.05, 0.1, 0.5] ,
        
    }
)
gs.fit(X_train, y_train, cat_features=cat_features)

In [ ]:
# gs1 = GridSearchCV(
#     LGBMRegressor(loss_function='MAE', random_seed=0),
#     param_grid={
#         'max_depth': [5, 7, 9],
#         'learning_rate': [0.05, 0.1, 0.5] ,
#     }
# )
# gs1.fit(X_train, y_train)

In [ ]:
gs.best_params_

{'learning_rate': 0.05, 'max_depth': 5}

In [ ]:
# gs1.best_params_

In [ ]:
mean_absolute_error(y_test, gs.predict(X_test))

In [ ]:
# mean_absolute_error(y_test, gs1.predict(X_test))

4

In [ ]:
import numpy as np

In [ ]:
cbr = CatBoostRegressor(learning_rate=0.05, max_depth=5)
cbr.fit(X_train, y_train) 

In [ ]:
def select_weights(y_true, y_pred_1, y_pred_2):
    metric = []
    grid = np.linspace(0, 1, 1000)
    for w_0 in grid:
      w_1 = 1 - w_0
      y = y_pred_1 * w_0 + y_pred_2 * w_1

      metric.append([mean_absolute_error(y_true, y), w_0, w_1])

    return metric

In [ ]:
mae_blending_train, w_0, w_1 = min(select_weights(y_train, gs.predict(X_train[num_features]), cbr.predict(X_test) ), key=lambda x: x[0])

In [ ]:
mae_blending_train 

In [ ]:
w_0, w_1 

(0.1831831831831832, 0.8168168168168168)

In [ ]:
mean_absolute_error(y_test, gs.predict(X_test[num_features]) * w_0 + cbr.predict(X_test) * w_1)

31.853400477728368

In [ ]:
xer = cbr.get_feature_importance()

In [ ]:
xer

In [ ]:
import seaborn as sns

features_imp = pd.DataFrame({'importance': xer,
'name': X_test.columns})

In [ ]:
sns.barplot(x='importance', y='name', data=features_imp)

X_train.drop(['floor', 'floors_total', 'rooms_offered', 'balcony', 'building_type', 'studio'], axis=1)
X_test.drop(['floor', 'floors_total', 'rooms_offered', 'balcony', 'building_type', 'studio'], axis=1)

Но по заданию те, что меньше 0.05, а эти минимум 0,5

In [ ]:
new_train = X_train.drop(['floor', 'floors_total', 'rooms_offered', 'balcony', 'building_type', 'studio'], axis=1)
new_test = X_test.drop(['floor', 'floors_total', 'rooms_offered', 'balcony', 'building_type', 'studio'], axis=1)

In [ ]:
new_cat = ['month', 'renovation']

In [ ]:
cbr1 = CatBoostRegressor(loss_function='MAE',learning_rate=0.05, max_depth=5)

In [ ]:
cbr1.fit(new_train, y_train, cat_features= new_cat)

In [ ]:
mean_absolute_error(y_test, cbr1.predict(new_test))

In [ ]:
mean_absolute_error(y_test, cbr.predict(new_test)) - mean_absolute_error(y_test, cbr1.predict(new_test))